In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/'My Drive'/'kaggle'/'LinkingWritingProcess'

/content/drive/My Drive/kaggle/LinkingWritingProcess


In [3]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 3.0 MB/s eta 0:00:00


In [4]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 9.9 MB/s eta 0:00:00


In [5]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('./data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import VarianceThreshold

import warnings
warnings.filterwarnings('ignore')
import datetime as dt

from catboost import CatBoostRegressor
import xgboost as xgb
import lightgbm as lgb

from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, RepeatedStratifiedKFold, cross_val_score
from sklearn.ensemble import HistGradientBoostingRegressor,ExtraTreesRegressor,GradientBoostingRegressor

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import optuna
from sklearn.ensemble import AdaBoostRegressor

from scipy.optimize import minimize

./data/sample_submission.csv
./data/test_logs.csv
./data/train_logs.csv
./data/train_scores.csv
./data/Essay Constructor Result.csv


## (1) データの読み込み

In [6]:
# 訓練データ読み込み
train_df = pd.read_csv("./data/train_logs.csv")
train_scores = pd.read_csv("./data/train_scores.csv")
# 検証データ読み込み
test_df = pd.read_csv("./data/test_logs.csv")

print("train_df： ", train_df.shape)
print("train_scores: ", train_scores.shape)
print("test_df: ", test_df.shape)

train_df：  (8405898, 11)
train_scores:  (2471, 2)
test_df:  (6, 11)


In [7]:
# 訓練データの表示確認
train_df.head()

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count
0,001519c8,1,4526,4557,31,Nonproduction,Leftclick,Leftclick,NoChange,0,0
1,001519c8,2,4558,4962,404,Nonproduction,Leftclick,Leftclick,NoChange,0,0
2,001519c8,3,106571,106571,0,Nonproduction,Shift,Shift,NoChange,0,0
3,001519c8,4,106686,106777,91,Input,q,q,q,1,1
4,001519c8,5,107196,107323,127,Input,q,q,q,2,1


In [9]:
train_scores.head()

,id,score
0,001519c8,3.5
1,0022f953,3.5
2,0042269b,6.0
3,0059420b,2.0
4,0075873a,4.0


In [8]:
# 検証データの表示確認
test_df.head()

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count
0,0000aaaa,1,338433,338518,85,Input,Space,Space,,0,0
1,0000aaaa,2,760073,760160,87,Input,Space,Space,,1,0
2,2222bbbb,1,711956,712023,67,Input,q,q,q,0,1
3,2222bbbb,2,290502,290548,46,Input,q,q,q,1,1
4,4444cccc,1,635547,635641,94,Input,Space,Space,,0,0


## (2) データ概要確認

In [10]:
# 各項目のデータ型の確認
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8405898 entries, 0 to 8405897
Data columns (total 11 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   id               object
 1   event_id         int64 
 2   down_time        int64 
 3   up_time          int64 
 4   action_time      int64 
 5   activity         object
 6   down_event       object
 7   up_event         object
 8   text_change      object
 9   cursor_position  int64 
 10  word_count       int64 
dtypes: int64(6), object(5)
memory usage: 705.5+ MB


In [11]:
# データの統計データを確認
train_df.describe()

,event_id,down_time,up_time,action_time,cursor_position,word_count
count,8.405898e+06,8.405898e+06,8.405898e+06,8.405898e+06,8.405898e+06,8.405898e+06
mean,2.067649e+03,7.935603e+05,7.936584e+05,9.808498e+01,1.222964e+03,2.314687e+02
std,1.588284e+03,5.149451e+05,5.149428e+05,2.533985e+02,9.485242e+02,1.759088e+02
min,1.000000e+00,1.060000e+02,2.520000e+02,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.520000e+02,3.731842e+05,3.732820e+05,6.600000e+01,4.990000e+02,9.600000e+01
50%,1.726000e+03,7.208860e+05,7.209800e+05,9.300000e+01,1.043000e+03,2.000000e+02
75%,2.926000e+03,1.163042e+06,1.163141e+06,1.220000e+02,1.706000e+03,3.270000e+02
max,1.287600e+04,8.313630e+06,8.313707e+06,4.474700e+05,7.802000e+03,1.326000e+03


In [12]:
# 欠損値の確認(訓練データ)
for col in train_df.columns:
    pct_missing = np.mean(train_df[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

id - 0%
event_id - 0%
down_time - 0%
up_time - 0%
action_time - 0%
activity - 0%
down_event - 0%
up_event - 0%
text_change - 0%
cursor_position - 0%
word_count - 0%


In [13]:
# 欠損値の確認(検証データ)
for col in test_df.columns:
    pct_missing = np.mean(test_df[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

id - 0%
event_id - 0%
down_time - 0%
up_time - 0%
action_time - 0%
activity - 0%
down_event - 0%
up_event - 0%
text_change - 0%
cursor_position - 0%
word_count - 0%


## (3) 特徴量の作成


https://www.kaggle.com/code/polyakovstepan/random-forest-atboost-xgb-baseline

In [14]:
def summary_time(df):
    result = df.groupby('id')['action_time'].sum().reset_index()
    result.rename(columns={'action_time': 'summary_time'}, inplace=True)
    return result

def start_pause(df):
    result = df.groupby('id')['down_time'].min().reset_index()
    result.rename(columns={'down_time':'start_pause'}, inplace=True)
    return result

def enter_click(df):
    copy_df = df
    copy_df['enter_click'] = (copy_df['down_event'] == 'Enter')
    copy_df = copy_df.groupby('id')['enter_click'].sum().reset_index()
    return copy_df

def space_click(df):
    copy_df = df
    copy_df['space_click'] = (copy_df['down_event'] == 'Space')
    copy_df = copy_df.groupby('id')['space_click'].sum().reset_index()
    return copy_df

def backspace_click(df):
    copy_df = df
    copy_df['backspace_click'] = (copy_df['down_event'] == 'Backspace')
    copy_df = copy_df.groupby('id')['backspace_click'].sum().reset_index()
    return copy_df

def symbol_length(df):
    result = df.groupby('id')['cursor_position'].max().reset_index()
    result.rename(columns={'cursor_position':'symbol_length'}, inplace=True)
    return result

def text_length(df):
    result = df.groupby('id')['word_count'].max().reset_index()
    return result

def nonproduction_feature(df):
    result = df.groupby('id')['activity'].apply(lambda x: (x == 'Nonproduction').mean() * 100).reset_index()
    result.rename(columns={'activity': 'nonproduction_feature'}, inplace=True)
    return result

def input_feature(df):
    result = df.groupby('id')['activity'].apply(lambda x: (x == 'Input').mean() * 100).reset_index()
    result.rename(columns={'activity': 'input_feature'}, inplace=True)
    return result

def remove_feature(df):
    result = df.groupby('id')['activity'].apply(lambda x: (x == 'Remove/Cut').mean() * 100).reset_index()
    result.rename(columns={'activity': 'remove_feature'}, inplace=True)
    return result

def mean_action_time(df):
    result = df.groupby('id')['action_time'].mean().reset_index()
    result.rename(columns={'action_time':'mean_action_time'}, inplace=True)
    return result

def replace_feature(df):
    result = df[df['activity'] == 'Replace'].groupby('id').size().reset_index(name='replace_feature')
    return result

def text_change_unique(df):
    result = df.groupby('id')['text_change'].nunique().reset_index()
    result.rename(columns={'text_change': 'tch_unique'}, inplace=True)
    return result

def sentence_size_feature(df):
    result = df[(df['text_change'] == '.') & (df['down_event'] != 'Backspace')].groupby('id').size().reset_index(name = 'number_sentence')
    return result

In [15]:
def getDataset(train_df):
    new_df = summary_time(train_df)

    functions = [
        start_pause,enter_click,space_click,backspace_click,symbol_length,text_length,
        nonproduction_feature, input_feature, remove_feature, mean_action_time, replace_feature,
        text_change_unique, sentence_size_feature
    ]

    for func in functions:
        result_df = func(train_df)
        new_df = pd.merge(new_df, result_df, on='id', how='outer')

    return new_df

In [16]:
# 訓練データの特徴量生成
df = getDataset(train_df)
print('train_info: ', df.shape)
df.head()

train_info:  (2471, 15)


,id,summary_time,start_pause,enter_click,space_click,backspace_click,symbol_length,word_count,nonproduction_feature,input_feature,remove_feature,mean_action_time,replace_feature,tch_unique,number_sentence
0,001519c8,297243,4526,4,357,417,1539,256,4.693000,78.607743,16.308174,116.246774,7.0,17,21.0
1,0022f953,275391,30623,6,391,260,1676,323,10.350448,78.973105,10.594947,112.221271,1.0,12,15.0
2,0042269b,421201,4441,17,552,439,2291,404,4.231141,84.985493,10.614120,101.837766,7.0,19,21.0
3,0059420b,189596,41395,3,243,152,1047,206,6.362468,83.804627,9.704370,121.848329,1.0,10,13.0
4,0075873a,313702,78470,10,324,517,1402,252,2.844725,76.728566,20.426709,123.943896,NaN,9,23.0


In [17]:
essay = pd.read_csv('./data/Essay Constructor Result.csv')
essay.head()

,id,essay
0,001519c8,qqqqqqqqq qq qqqqq qq qqqq qqqq. qqqqqq qqq q...
1,0022f953,"qqqq qq qqqqqqqqqqq ? qq qq qqq qqq qqq, qqqqq..."
2,0042269b,qqqqqqqqqqq qq qqqqq qqqqqqqqq qq qqqqqqqqqqq ...
3,0059420b,qq qqqqqqq qqqqqq qqqqqqqqqqqqq qqqq q qqqq qq...
4,0075873a,"qqqqqqqqqqq qq qqq qqqqq qq qqqqqqqqqq, qqq qq..."


In [18]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

def essay_difficulty(df):
    df['token'] = [word_tokenize(p) for p in df["essay"]]
    df['token_len'] = df['token'].apply(lambda x : list(len(word) for word in x))
    df['verylong']  = df['token_len'].apply(lambda x : sum(c>=9 for c in x))
    df['long']      = df['token_len'].apply(lambda x : sum(c==7 or c==8 for c in x))
    df['mid']       = df['token_len'].apply(lambda x : sum(c==5 or c==6 for c in x))
    df['difficulty'] = df['verylong']*5 + df['long']*3 + df['mid']*1

    df['long_words'] = df['verylong']+df['long']

    return df

essay = essay_difficulty(essay)
df = pd.merge(df, essay[["id","difficulty", "long_words"]], on ="id", how="left")
df.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


,id,summary_time,start_pause,enter_click,space_click,backspace_click,symbol_length,word_count,nonproduction_feature,input_feature,remove_feature,mean_action_time,replace_feature,tch_unique,number_sentence,difficulty,long_words
0,001519c8,297243,4526,4,357,417,1539,256,4.693000,78.607743,16.308174,116.246774,7.0,17,21.0,312,64
1,0022f953,275391,30623,6,391,260,1676,323,10.350448,78.973105,10.594947,112.221271,1.0,12,15.0,179,30
2,0042269b,421201,4441,17,552,439,2291,404,4.231141,84.985493,10.614120,101.837766,7.0,19,21.0,597,131
3,0059420b,189596,41395,3,243,152,1047,206,6.362468,83.804627,9.704370,121.848329,1.0,10,13.0,190,40
4,0075873a,313702,78470,10,324,517,1402,252,2.844725,76.728566,20.426709,123.943896,NaN,9,23.0,216,44


In [20]:
def revealing_text(df):
    string = '' # empty string to add or edit the input sequence
    cursor = 0 # position of cursor
    shift_timer = 0 # Recording `up_time` for `Shift` key if `down_time` != `up_time`
    decoded_num = 0 # Counting the number of decoded char
    INSERT = False # Switch for `Insert` key.
    CAPSLOCK = False # Switch for `CapsLock` key.
    SHIFT = False # Switch for `Shift` key

    '''
    This function converts the list of input sequence to a full essay.
    - OCT 21 2023 Still in progress

    Thank you @kawaiicoderuwu for thoughts on Shift, however... it didn't work...
    I have still implemented the idea.
    Shift reconstructed based on:
    https://www.kaggle.com/competitions/linking-writing-processes-to-writing-quality/discussion/447735#2487004

    '''
    # Checking whether the reconstruction was done well with comparing the `text_change` column.
    YES = 0
    NO = 0
    sp_character_dict = {';':':',
                         "'": '"',
                        ',':'<',
                        '.':'>',
                        '/':'?',
                        '\\': '|',
                         '1':'!',
                         '2':'@',
                         '3':'#',
                         '4':'$',
                         '5':'%',
                         '6':'^',
                         '7':'&',
                         '8':'*',
                         '9':'(',
                         '0':')',
                         '-':'_',
                         '=':'+',
                         '`':'~',
                         '[':'{',
                         ']':'}'
                        }


    def sentence_splitter(string, char_per_line=140):
        '''
        This function will separate the sentence based on the size of prompt.

        Input :
        string : Sentence
        char_per_line : Number of characters in one line (not sentence).
        Output :
        List of splitted sentences.

        I have manually counted the number of characters per line.
        Based on the prompt depicted in Competition Overview - Data Collection Procedure.
        www.kaggle.com/competitions/linking-writing-processes-to-writing-quality/overview/data-collection-procedure
        '''
        each = []
        remaining_words = len(string) % char_per_line
        full_line = len(string) // char_per_line
        crsr = 0
        for i in range(full_line):
            if crsr == 0:
                each.append(string[:char_per_line])
                crsr = char_per_line
            else:
                each.append(string[crsr:(i+1)*char_per_line])
                crsr = crsr + char_per_line
        each.append(string[:-len(string)%char_per_line +1])
        return each

    def enter_splitter(list_line):
        '''
        This function will split the sentence if it contains \n (Enter) into 2 separate string.

        Input :
        list_line : Sentences in list (connected with `sentence_splitter` function in this notebook.)
        Output :
        enter_splitted : List of splitted sentence.
        '''
        enter_splitted = []
        for line in list_line:
            if '\n' in line:
                for lin in line.split('\n'):
                    if lin != line.split('\n')[-1]:
                        enter_splitted.append(lin+'\n')
                    else:
                        enter_splitted.append(lin)
            else:
                enter_splitted.append(line)
        return enter_splitted

    for row_num in range(df.shape[0]): # iterates over each row in df.
        # I was lazy to type `df['down_event'].iloc[row_num]` all the time.
        action = df['decoded_down_event'].iloc[row_num]
        if action in useless:
            continue
        if action == ' ':
            decoded_num = decoded_num + 1
        # I used `df['cursor_position'].iloc[row_num]` everywhere in the code
        # and I think not putting this in a variable was the problem.
        cursor_now = df['cursor_position'].iloc[row_num]

        # Config... kinda?

        if action == 'Insert':
            if INSERT == True:
                INSERT = False
                continue
            else:
                INSERT = True
                continue

        elif action == 'CapsLock':
            if CAPSLOCK == True:
                CAPSLOCK = False
                continue
            else:
                CAPSLOCK = True
                continue

        if action == 'Shift' and (df['down_time'].iloc[row_num] != df['up_time'].iloc[row_num]):
            shift_timer = df['up_time'].iloc[row_num]
            SHIFT = True
            continue

        if shift_timer != 0:
            shift_timer = 0
            SHIFT = False

        '''
        I have added two if-statements above to be prepared for the occasion
        where `down_time` and `up_time` is different for `Shift`.
        As all `down_time` and `up_time` are the same in training data,
        these conditions wouldn't be activated.
        '''


        if shift_timer == 0:
            if action == 'Shift':
                SHIFT = True
                continue

            elif df['decoded_down_event'].iloc[row_num-2] == 'Shift':
                SHIFT = False

        if row_num == 0:
            cursor = 0

        else:
            cursor = df['cursor_position'].iloc[row_num-1]


        ################################################################
        ################################################################

        # Cursor position


        if (action == 'Tab')&(row_num!=0):
            if df['decoded_down_event'].iloc[row_num-1] == 'Alt':
                '''
                Ignores `Alt + Tab`, which is the window switching keyboard shortcut.
                Not sure if it is in the sequence, but I just added it.
                '''
                continue


            if cursor == len(string):
                '''
                Adding `\t` for `Tab` at the end of the string.
                '''
                string = string + '\t'
                cursor = cursor_now
                continue


            else:
                '''
                Adding `\t` for `Tab` where the cursor is positioned at.
                '''
                string = string[:cursor] + '\t' + string[cursor:]
                cursor = cursor_now
                continue


        ################################################################
        ################################################################

        # Remove/Cut
        if df['activity'].iloc[row_num] == 'Remove/Cut':
            '''
            For Backspace & Delete, we can only consider when the activity == 'Remove/Cut'.
            '''
            if action == 'Backspace':

                # When the cursor is at the end of the string.
                if cursor == len(string):
                    # Simply removes the last character.
                    string = string[:-1]
                    # The cursor repositions as the length of the string decreases.
                    cursor = cursor_now
                    continue

                elif cursor == 0:
                    '''
                    There is no point of `Backspace` when the cursor is at the beginning of the whole essay.
                    '''
                    cursor = cursor_now
                    continue
                else:
                    '''
                    When the cursor is in the middle of the string.
                    Removes the last character before the cursor position and add the remaining string.
                    '''
                    string = string[:cursor-1] + string[cursor:]
                    # The cursor repositions as the length of the string decreases.
                    cursor = cursor_now
                    continue

            elif action == 'Delete':
                '''
                There is no point of `Delete` when the cursor is at the end of the sentence.
                '''
                if cursor == len(string):
                    cursor = cursor_now
                    continue

                # When the cursor is positioned at the beginning of the essay.
                elif cursor == 0:
                    '''
                    Simply deletes the first character of the whole essay.
                    '''
                    string = string[1:]
                    cursor = cursor_now
                    continue

                else:
                    '''
                    When the cursor is in the middle of the sentence.
                    A character right after the cursor position is removed,
                    then it is added to the string before the cursor.
                    '''
                    string = string[:cursor] + string[cursor+1:]
                    '''
                    Cursor doesn't move, check the discussion below.
                    https://www.kaggle.com/competitions/linking-writing-processes-to-writing-quality/discussion/448180#2488038
                    '''
                    cursor = cursor_now
                    continue

        ################################################################
        ################################################################

        elif df['activity'].iloc[row_num].startswith('Move'):
            act = df['activity'].iloc[row_num].split()
            before_start = int(act[2][1:-1])
            before_end = int(act[3][0:-1])
            after_start = int(act[5][1:-1])
            after_end = int(act[6][0:-1])

            string_to_be_moved = string[before_start:before_end]
            string = string[:before_start] + string[before_end:after_start] + string_to_be_moved + string[after_end:]
            cursor = cursor_now
            continue

        ################################################################
        ################################################################

        elif df['activity'].iloc[row_num] == 'Paste':
            string = string[:cursor] + df['decoded_after_change'].iloc[row_num] + string[cursor:]
            cursor = cursor_now
            continue

        ################################################################
        ################################################################

        elif df['activity'].iloc[row_num] == 'Input':
            input_ = df['decoded_after_change'].iloc[row_num]
            if input_ in list_of_inputs:
                # When the `Insert` switch is off.
                if INSERT == False:
                    # When the `CapsLock` switch is off.
                    if CAPSLOCK == False:
                        # When the cursor is at the end of the string.
                        if cursor == len(string):
                            # When Shift switch is off
                            if SHIFT == False:
                                '''
                                The input is simply added at the end of the string.
                                '''
                                string = string + input_
                                cursor = cursor_now
                                continue

                            else: # When Shift switch is on.
                                if input_.isalpha():
                                    '''
                                    The capitalized input is added at the end of the string.
                                    '''
                                    string = string + input_.upper()
                                    cursor = cursor_now
                                    continue

                                elif input_ in sp_character_dict.keys():
                                    '''
                                    The keys are removed, replaced with the values. (Based on my keyboard)
                                    But I think the values are already in the sequence.
                                    '''
                                    string = string + sp_character_dict.get(input_)
                                    cursor = cursor_now
                                    continue

                                else:
                                    string = string + input_
                                    cursor = cursor_now
                                    continue

                        else:
                            '''
                            When the cursor is not at the end of the string.
                            '''
                            if SHIFT == False:
                                '''
                                The input is added at the cursor position.
                                '''
                                string = string[:cursor] + input_ + string[cursor:]
                                cursor = cursor_now
                                continue
                            else:
                                if input_.isalpha():
                                    string = string[:cursor] + input_.upper() + string[cursor:]
                                    cursor = cursor_now
                                    continue

                                elif input_ in sp_character_dict.keys():
                                    string = string[:cursor] + sp_character_dict.get(input_)  + string[cursor:]
                                    cursor = cursor_now
                                    continue

                                else:
                                    string = string[:cursor] + input_ + string[cursor:]
                                    cursor = cursor_now
                                    continue
                    else:
                        '''
                        When `CapsLock` switch is on.
                        When the cursor is at the end of the essay.
                        '''
                        if cursor == len(string):
                            if input_.isalpha():
                                '''
                                The alphabets typed after this loop will be in capitals until the CapsLock is turned off.
                                '''
                                string = string + input_.upper()
                                cursor = cursor_now
                                continue

                            else:
                                '''
                                As the `CapsLock` is only applied to the alphabets,
                                no special action is need for non-alphabets.
                                '''
                                string = string + input_
                                cursor = cursor_now
                                continue

                        else:
                            '''
                            When the cursor is positioned in the middle of the string.
                            A caplitalized alphabet is added at the cursor position.
                            '''
                            string = string[:cursor] + input_.upper() + string[cursor:]
                            cursor = cursor_now
                            continue


                else: # When `Insert` switch is on.
                    '''
                    I will skip the explanations for `CapsLock` and focus more on `Insert`
                    '''
                    if CAPSLOCK == False:
                        if cursor == len(string):
                            if SHIFT == False:
                                string = string + input_
                                cursor = cursor_now
                                continue

                            else:
                                if input_.isalpha():
                                    string = string + input_.upper()
                                    cursor = cursor_now
                                    continue

                                elif input_ in sp_character_dict.keys():
                                    string = string + sp_character_dict.get(input_)
                                    cursor = cursor_now
                                    continue

                                else:
                                    string = string + input_
                                    cursor = cursor_now
                                    continue

                        else: # When the cursor is in the middle of the string.
                            if SHIFT == False:
                                string = string[:cursor] + input_ + string[cursor+1:]
                                cursor = cursor_now
                                continue

                            else:
                                if input_.isalpha()==True:
                                    string = string[:cursor] + input_.upper() + string[cursor+1:]
                                    cursor = cursor_now
                                    continue

                                elif input_ in sp_character_dict.keys():
                                    string = string[:cursor] + sp_character_dict.get(input_) + string[cursor+1:]
                                    cursor = cursor_now
                                    continue

                                else:
                                    string = string[:cursor]+ input_ + string[cursor+1:]
                                    cursor = cursor_now
                                    continue

                            '''
                            It takes full string before the cursor position
                            Add input
                            Then add removes the first character of the remaining string and add it to the previous string.
                            This logic applies to the whole `else` statement, so I will skip.
                            '''

                    else: # When CapsLock is on.
                        if cursor == len(string):
                            if input_.isalpha():
                                string = string + input_.upper()
                                cursor = cursor_now
                                continue

                            else: # When the action is not alphabet.
                                if SHIFT==False:
                                    string = string + input_
                                    cursor = cursor_now
                                    continue

                                elif input_ in sp_character_dict.keys():
                                    string = string + sp_character_dict.get(input_)
                                    cursor = cursor_now
                                    continue
                                else:
                                    string = string + action
                                    cursor = cursor_now
                                    continue
                        else:
                            if input_.isalpha()==True:
                                string = string[:cursor] + input_.upper() + string[cursor+1:]
                                cursor = cursor_now
                                continue

                            else:
                                if SHIFT==False:
                                    string = string[:cursor] + input_ + string[cursor+1:]
                                    continue

                                elif input_ in sp_character_dict.keys():
                                    string = string[:cursor] + sp_character_dict.get(input_) + string[cursor+1:]
                                    cursor = cursor_now
                                    continue

                                else:
                                    string = string[:cursor] + input_ + string[cursor+1:]
                                    cursor = cursor_now
                                    continue

            ################################################################

            if action == 'Space': # When the `Space` was inserted
                if cursor == len(string):
                    string = string + input_
                    cursor = cursor_now
                    continue

                else:
                    if INSERT == False:
                        '''
                        When `Insert` switch is off and the cursor is not positioned at the end of the sentence.
                        '''
                        string = string[:cursor] + input_ + string[cursor:]
                        cursor = cursor_now
                        continue

                    else:
                        '''
                        When `Insert` switch is on and the cursor is not positioned at the end of the sentence.
                        '''
                        string = string[:cursor] + input_ + string[cursor+1:]
                        cursor = cursor_now
                        continue

            ################################################################

            elif action == 'Enter': # When the `Enter` was inserted
                if cursor == len(string):
                    string = string + input_
                    cursor = cursor_now
                    continue

                else:
                    string = string[:cursor] + input_ + string[cursor:]
                    cursor = cursor_now
                    continue

        ################################################################
        ################################################################


        elif (df['activity'].iloc[row_num] == 'Replace') & (df['decoded_before_change'].iloc[row_num]!='-'):
            '''
            When the `activity` column is set as `Replace`.
            '''
            before = df['decoded_before_change'].iloc[row_num].strip() # string to be replaced.
            after = df['decoded_after_change'].iloc[row_num].strip() # string replaced.

            string = string[:cursor-len(before)-1] + after + string[cursor:]
            cursor = cursor_now
            continue

    if len(string.split()) - decoded_num == df['word_count'].iloc[row_num]:
        '''
        Check if the replace was well-performed.
        Which most probably means my reconstruction worked well.
        '''
        YES = YES+1
    else:
        '''
        My reconstruction didn't work well.... omg... Need to work harder.
        '''
        NO = NO+1
    return string, YES, NO

In [21]:
train_revealed = pd.DataFrame({'id':[], 'essay':[]})
# counter = 0
YESSSS = 0
NOOOOO = 0
for i in test_df['id'].unique().tolist():

    df = test_df[test_df['id']==i]
    df_string, YES, NO = revealing_text(df)
    YESSSS = YESSSS + YES
    NOOOOO = NOOOOO + NO
    temp_df = pd.DataFrame({'id':[i], 'essay':[df_string]})
    train_revealed = pd.concat([train_revealed, temp_df], axis=0)

try:
    yes_no_ratio = YESSSS/(YESSSS+NOOOOO)
    print(f'Success Rate = {yes_no_ratio:.2f}')

except:
    pass

display(train_revealed.iloc[0])

KeyError: ignored

In [19]:
# 検証データの特徴量生成
test = getDataset(test_df)
test = pd.merge(test, essay[["id","difficulty", "long_words"]], on ="id", how="left")
print('test_info: ', test.shape)
test.head(10)

test_info:  (3, 17)


,id,summary_time,start_pause,enter_click,space_click,backspace_click,symbol_length,word_count,nonproduction_feature,input_feature,remove_feature,mean_action_time,replace_feature,tch_unique,number_sentence,difficulty,long_words
0,0000aaaa,172,338433,0,2,0,1,0,0.0,100.0,0.0,86.0,NaN,1,NaN,NaN,NaN
1,2222bbbb,113,290502,0,0,0,1,1,0.0,100.0,0.0,56.5,NaN,1,NaN,NaN,NaN
2,4444cccc,150,184996,0,1,0,1,1,0.0,100.0,0.0,75.0,NaN,2,NaN,NaN,NaN
